<a href="https://colab.research.google.com/github/yeqinghuang516/Auto-Drive/blob/master/DeepPicar_3_Camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip

--2020-03-03 00:45:35--  https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 13.227.209.219, 13.227.209.224, 13.227.209.199, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|13.227.209.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 333137665 (318M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 317.70M  51.6MB/s    in 6.5s    

2020-03-03 00:45:42 (49.2 MB/s) - ‘data.zip’ saved [333137665/333137665]



In [0]:
!unzip data.zip

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import csv
import cv2
import torch
import torch.utils.data as data
import numpy as np

samples = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for line in reader:
        samples.append(line)
        

train_len = int(0.8*len(samples))
valid_len = len(samples) - train_len
train_samples, validation_samples = data.random_split(samples, lengths=[train_len, valid_len])

In [0]:
def augment(imgName, angle):
  name = 'data/IMG/' + imgName.split('/')[-1]
  current_image = cv2.imread(name)
  current_image = current_image[65:-25, :, :]
  if np.random.rand() < 0.5:
    current_image = cv2.flip(current_image, 1)
    angle = angle * -1.0  
  return current_image, angle

In [0]:
class Dataset(data.Dataset):

    def __init__(self, samples, transform=None):

        self.samples = samples
        self.transform = transform

    def __getitem__(self, index):
      
        batch_samples = self.samples[index]
        
        steering_angle = float(batch_samples[3])
        
        center_img, steering_angle_center = augment(batch_samples[0], steering_angle)
        left_img, steering_angle_left = augment(batch_samples[1], steering_angle + 0.2)
        right_img, steering_angle_right = augment(batch_samples[2], steering_angle - 0.2)

        center_img = self.transform(center_img)
        left_img = self.transform(left_img)
        right_img = self.transform(right_img)

        return (center_img, steering_angle_center), (left_img, steering_angle_left), (right_img, steering_angle_right)
      
    def __len__(self):
        return len(self.samples)
      

In [0]:
params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 4}

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])

training_set = Dataset(train_samples, transformations)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(validation_samples, transformations)
validation_generator = data.DataLoader(validation_set, **params)


In [0]:
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F

class NetworkDense(nn.Module):
  # DeepPicar by NVIDIA

    def __init__(self):
        super(NetworkDense, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ELU(),
            # nn.MaxPool2d(2),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            # nn.MaxPool2d(2),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            # nn.MaxPool2d(2),
            nn.Conv2d(48, 64, 3),
            nn.ELU(),
            nn.Conv2d(64, 64, 3),
            nn.Dropout(0.25)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64 * 2 * 33, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, input):  
        input = input.view(input.size(0), 3, 70, 320)
        output = self.conv_layers(input)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output

In [0]:
model = NetworkDense()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.MSELoss()

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print('device is: ', device)

device is:  cuda


In [0]:
def toDevice(datas, device):
  imgs, angles = datas
  return imgs.float().to(device), angles.float().to(device)

In [0]:
from pytorchtools import EarlyStopping

In [0]:
from tqdm import tqdm
max_epochs = 50
train_losses = []
valid_losses = []
early_stopping = EarlyStopping(patience=5, verbose=True)

for epoch in tqdm(range(max_epochs)):
    print('Epoch {}'.format(epoch))
    model.to(device)
    
    # Training
    train_loss = 0
    model.train()
    for local_batch, (centers, lefts, rights) in enumerate(training_generator):
        # Transfer to GPU
        centers, lefts, rights = toDevice(centers, device), toDevice(lefts, device), toDevice(rights, device);
        
        # Model computations
        optimizer.zero_grad()
        datas = [centers, lefts, rights]
        # datas = [centers]
        for data in datas:
            imgs, angles = data
            outputs = model(imgs)
            loss = criterion(outputs, angles.unsqueeze(1))
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item();
        avg_train_loss = train_loss/((local_batch+1)*3)

    
    # Validation
    model.eval()
    valid_loss = 0
    with torch.set_grad_enabled(False):
        for local_batch, (centers, lefts, rights) in enumerate(validation_generator):
            # Transfer to GPU
            centers, lefts, rights = toDevice(centers, device), toDevice(lefts, device), toDevice(rights, device)
        
            # Model computations
            optimizer.zero_grad()
            datas = [centers, lefts, rights]    
            # datas = [centers]  
            for data in datas:
                imgs, angles = data
                outputs = model(imgs)
                loss = criterion(outputs, angles.unsqueeze(1))
                
                valid_loss += loss.data.item()

            avg_valid_loss = valid_loss/((local_batch+1)*3)
    train_losses.append(avg_train_loss)
    valid_losses.append(avg_valid_loss)
    state = {
        'model': model.module if device == 'cuda' else model,
         'epoch': epoch,
         'model_state_dict': model.state_dict(),
         'optimizer_state_dict': optimizer.state_dict(),
         'train loss': train_losses,
         'valid loss': valid_losses
        }
    early_stopping(valid_loss, state)

    if early_stopping.early_stop:
        print("Early stopping")
        break

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 0


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type NetworkDense. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  2%|▏         | 1/50 [00:45<37:03, 45.38s/it]

Validation loss decreased (inf --> 3.669714).  Saving model ...
Epoch 1


  4%|▍         | 2/50 [01:20<33:51, 42.32s/it]

...

EarlyStopping counter: 2 out of 5
Epoch 27


 56%|█████▌    | 28/50 [16:00<12:11, 33.24s/it]

EarlyStopping counter: 3 out of 5
Epoch 28


 58%|█████▊    | 29/50 [16:33<11:40, 33.34s/it]

EarlyStopping counter: 4 out of 5
Epoch 29
EarlyStopping counter: 5 out of 5
Early stopping


In [0]:
!cp bestModel.h5 /content/drive/My\ Drive/Autonomous\ Driving/deepPicar_3camera.h5

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))
plt.plot(train_losses, label='training loss')
plt.plot(valid_losses, label='validation loss')
plt.grid()
plt.legend(prop={'size': 20})
plt.xlabel('Epoch', fontsize = 16);
plt.ylabel('Average Sample Loss', fontsize = 16);
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

NameError: ignored

<Figure size 1080x720 with 0 Axes>

In [0]:
min(valid_losses)

0.013680304557365042